In [1]:
from email.parser import BytesParser

In [2]:
ls 

SpamDetector.ipynb  easy_ham/           spam/


In [3]:
SPAM_PATH='.'

In [4]:

import email
import email.policy

def load_email(is_spam, filename, spam_path=SPAM_PATH):
    directory = "spam" if is_spam else "easy_ham"
    with open(os.path.join(spam_path, directory, filename), "rb") as f:
        return BytesParser(policy=email.policy.default).parse(f)

In [5]:
import os

In [7]:
easy_ham_emails = [ load_email(False,email) for email in os.listdir('easy_ham/')]

In [8]:
spam_emails = [ load_email(True,email) for email in os.listdir('spam/')]

In [25]:
em = easy_ham_emails[0]
for key in em.keys():
    print(key,em[key])

Return-Path <rssfeeds@example.com>
Delivered-To yyyy@localhost.example.com
Received from localhost (jalapeno [127.0.0.1])	by jmason.org (Postfix) with ESMTP id AE79816F16	for <jm@localhost>; Mon, 30 Sep 2002 13:43:46 +0100 (IST)
Received from localhost (jalapeno [127.0.0.1])	by jmason.org (Postfix) with ESMTP id AE79816F16	for <jm@localhost>; Mon, 30 Sep 2002 13:43:46 +0100 (IST)
Received from localhost (jalapeno [127.0.0.1])	by jmason.org (Postfix) with ESMTP id AE79816F16	for <jm@localhost>; Mon, 30 Sep 2002 13:43:46 +0100 (IST)
Message-Id <200209300801.g8U81fg21359@dogma.slashnull.org>
To yyyy@example.com
From gamasutra <rssfeeds@example.com>
Subject Priceless Rubens works stolen in raid on mansion
Date Mon, 30 Sep 2002 08:01:41 -0000
Content-Type text/plain; encoding="utf-8"
Lines 6
X-Spam-Status No, hits=-527.4 required=5.0	tests=AWL,DATE_IN_PAST_03_06,T_URI_COUNT_0_1	version=2.50-cvs
X-Spam-Level 


In [42]:
import re
from html import unescape

def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text)

In [142]:

def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ("text/plain", "text/html"):
            continue
        try:
            content = part.get_content()
        except: # in case of encoding issues
            content = str(part.get_payload())
        if ctype == "text/plain":
            return content
        else:
            html = content
    if html:
        return html_to_plain_text(html)
    else:
        return ''

In [48]:
!pip3 install urlextract

In [50]:
import urlextract

In [55]:
X = spam_emails + easy_ham_emails

In [61]:
y = [0] * len(spam_emails) + [1] * len(easy_ham_emails) 

In [65]:
from sklearn.model_selection import train_test_split

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [73]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer

In [78]:
import string
string.punctuation
table = str.maketrans('', '', string.punctuation)
table
[ w.translate(table) for w in 'Hi i"m kunal . do you know me ? '.split()]

['Hi', 'im', 'kunal', '', 'do', 'you', 'know', 'me', '']

In [143]:
class EmailToWordCounter(BaseEstimator, TransformerMixin):
    
    def __init__(self,lowercase=True, remove_punctuation=True, replace_url=True, replace_number=True):
        self.lowercase=lowercase
        self.remove_punctuation=remove_punctuation
        self.replace_url=replace_url
        self.replace_number=replace_number
        self.url_extractor = urlextract.URLExtract()
        self.count_vectorizer = CountVectorizer()
        
        
    def do_remove_puctuation(self,text):
        import string
        table = str.maketrans('', '', string.punctuation)
        vector = [ w.translate(table) for w in text.split()]
        vector = [ word for word in vector if word != '' ]
        return ' '.join(vector)
    
    def do_replace_numbers(self, text):
        text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
        return text
    
    def do_replace_url(self, text):
        urls = self.url_extractor.find_urls(text)

        for url in urls:
            text = text.replace(url, 'URL')
            
        return text
    
    def preprocess(self,X):
        new_X=[]
        for email in X:
            email_text=email_to_text(email)
              
            if self.lowercase:email_text = email_text.lower()
                
            if self.replace_url:email_text=self.do_replace_url(email_text)
                
            if self.replace_number:email_text=self.do_replace_numbers(email_text)
                
            if self.remove_punctuation:email_text=self.do_remove_puctuation(email_text)
            
            new_X.append(email_text)
        
        return new_X
        
    def fit(self, X, y=None):
                    
        new_X = self.preprocess(X)
        
        self.count_vectorizer.fit(new_X) 
        
        return self
        
    def transform(self, X, y=None):
        
        new_X = self.preprocess(X)
        
        return self.count_vectorizer.transform(new_X)

In [144]:
email_to_word_transformer = EmailToWordCounter()
X_train_transformed=email_to_word_transformer.fit_transform(X_train)

In [145]:
from sklearn.linear_model import LogisticRegression


lr_reg = LogisticRegression()
lr_reg.fit(X_train_transformed, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [146]:
y_pred= lr_reg.predict(X_train_transformed)

In [147]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [148]:
confusion_matrix(y_train, y_pred)

array([[ 360,    2],
       [   0, 1927]])

In [149]:
X_test_transformed=email_to_word_transformer.transform(X_test)

In [150]:
X_test_transformed

<763x31179 sparse matrix of type '<class 'numpy.int64'>'
	with 82650 stored elements in Compressed Sparse Row format>

In [151]:
y_pred = lr_reg.predict(X_test_transformed)

In [152]:
accuracy_score(y_test,y_pred)

0.9855832241153342

In [153]:
confusion_matrix(y_test,y_pred)

array([[130,   9],
       [  2, 622]])

In [154]:
from sklearn.metrics import f1_score

In [155]:
f1_score(y_test, y_pred)

0.9912350597609562

In [157]:
from sklearn.pipeline import Pipeline

predictor = Pipeline([
    ('preparator', EmailToWordCounter()),
    ('predictor', LogisticRegression())
])

In [158]:
predictor.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('preparator', EmailToWordCounter(lowercase=True, remove_punctuation=True,
          replace_number=True, replace_url=True)), ('predictor', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [159]:
y_pred=predictor.predict(X_test)

In [162]:
from sklearn.metrics import precision_score,recall_score

In [163]:
precision_score(y_test, y_pred)

0.9857369255150554

In [164]:
recall_score(y_test, y_pred)

0.9967948717948718